`# Import dependencies`

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr
import matplotlib
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pio.renderers.default = 'iframe'
pd.options.plotting.backend = 'plotly'

`# Get stock market data`

In [2]:
end = dt.datetime.now()
start = dt.datetime(2015,1,1)

yf.pdr_override()
df = pdr.get_data_yahoo(["NVDA", "GOOG", "META", "TSLA"], start, end)
Close = df.Close
Close.head()

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


[*********************100%%**********************]  4 of 4 completed


Ticker,GOOG,META,NVDA,TSLA
Date,,,,
2015-01-02,26.168653,78.449997,0.50325,14.620667
2015-01-05,25.623152,77.190002,0.49475,14.006000
2015-01-06,25.029282,76.150002,0.47975,14.085333
2015-01-07,24.986401,76.150002,0.47850,14.063333
2015-01-08,25.065184,78.180000,0.49650,14.041333


`# Compute log returns`

In [3]:
log_returns = np.log(df.Close/df.Close.shift(1)).dropna()

`# Calculate daily standard deviation of returns`

In [4]:
daily_std = log_returns.std()
daily_std

Ticker
GOOG    0.017929
META    0.024101
NVDA    0.030254
TSLA    0.035467
dtype: float64

`# Compute annualized volatility`

In [5]:
annualized_vol = daily_std * np.sqrt(252)
annualized_vol*100

Ticker
GOOG    28.461731
META    38.259590
NVDA    48.027502
TSLA    56.301910
dtype: float64

`# Plot histogram of log returns with annualized volatility`

In [6]:
fig = make_subplots(rows = 2, cols = 2)

trace0 = go.Histogram(x = log_returns['GOOG'], name = 'GOOG')
trace1 = go.Histogram(x = log_returns['META'], name = 'META')
trace2 = go.Histogram(x = log_returns['NVDA'], name = 'NVDA')
trace3 = go.Histogram(x = log_returns['TSLA'], name = 'TSLA')

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)

fig.update_layout(title = 'Frequency of log returns',
                  xaxis = dict(title = 'GOOG Annualized Vol: ' + str(np.round(annualized_vol['GOOG']*100,1))), 
                  xaxis2 = dict(title = 'META Annualized Vol: ' + str(np.round(annualized_vol['META']*100,1))),
                  xaxis3 = dict(title = 'NVDA Annualized Vol: ' + str(np.round(annualized_vol['NVDA']*100,1))),
                  xaxis4 = dict(title = 'TSLA Annualized Vol: ' + str(np.round(annualized_vol['TSLA']*100,1))),
                 )

fig.show()

`# Trailing volatility over time`

In [7]:
TRADING_DAYS = 60
volatility = log_returns.rolling(window = TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)

In [8]:
volatility.plot()

`# Sharpe ratio`

In [9]:
Rf = 0.01/252
sharpe_ratio = (log_returns.rolling(window = TRADING_DAYS).mean() - Rf)*TRADING_DAYS/volatility
sharpe_ratio.plot()

`# Sortino Ratio`

In [10]:
sortino_vol = log_returns[log_returns < 0].rolling(window = TRADING_DAYS, min_periods = 15).std()*np.sqrt(TRADING_DAYS)
sortino_vol.plot()

In [11]:
sortino_ratio = (log_returns.rolling(window = TRADING_DAYS).mean() - Rf)*TRADING_DAYS/sortino_vol
sortino_ratio.plot()

`# Modiligliani ratio (M2 ratio)`

In [12]:
m2_ratio = pd.DataFrame()

benchmark_vol = volatility['GOOG']
for c in log_returns.columns:
    if c != 'GOOG':
        m2_ratio[c] = (sharpe_ratio[c]*benchmark_vol/TRADING_DAYS + Rf)*TRADING_DAYS

m2_ratio.plot()

`# Max Drawdown`

In [13]:
def max_drawdown(returns):
    cumulative_returns = (1 + returns).cumprod()
    peak = cumulative_returns.expanding(min_periods = 1).max()
    drawdown = (cumulative_returns/peak) - 1
    return drawdown.min()

returns = df.Close.pct_change().dropna()
max_drawdowns = returns.apply(max_drawdown, axis = 0)
max_drawdowns*100    

Ticker
GOOG   -44.601850
META   -76.736092
NVDA   -66.362055
TSLA   -73.632217
dtype: float64

`# Calmar ratio`

In [14]:
calmars = np.exp(log_returns.mean()*252)/abs(max_drawdowns) 
calmars.plot.bar()